In [21]:
import io
import numpy as np
import random
import sys
import time
from keras.utils.data_utils import get_file
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.models import load_model

In [3]:
print("Apertura del file")
with open('divinacommedia.txt', 'r') as f:
    text = f.read().lower()
print('Lunghezza del testo:', len(text))
print()
print('\n\n\n\n\n','*****************primi mille caratteri************\n',
      text[0:1000],'***********************************************\n')    

Apertura del file
Lunghezza del testo: 558240






 *****************primi mille caratteri************
 inferno



inferno: canto i


nel mezzo del cammin di nostra vita
  mi ritrovai per una selva oscura
  che' la diritta via era smarrita.

ahi quanto a dir qual era e` cosa dura
  esta selva selvaggia e aspra e forte
  che nel pensier rinova la paura!

tant'e` amara che poco e` piu` morte;
  ma per trattar del ben ch'i' vi trovai,
  diro` de l'altre cose ch'i' v'ho scorte.

io non so ben ridir com'i' v'intrai,
  tant'era pien di sonno a quel punto
  che la verace via abbandonai.

ma poi ch'i' fui al pie` d'un colle giunto,
  la` dove terminava quella valle
  che m'avea di paura il cor compunto,

guardai in alto, e vidi le sue spalle
  vestite gia` de' raggi del pianeta
  che mena dritto altrui per ogne calle.

allor fu la paura un poco queta
  che nel lago del cor m'era durata
  la notte ch'i' passai con tanta pieta.

e come quei che con lena affannata
  uscito fuor del pelago a la r

In [4]:
# preparo i dati per ambientare la rete neurale

chars = sorted (list (set (text)))
char_indices = dict((c, i) for i, c in enumerate(chars)) # valore - numero
indices_char = dict((i, c) for i, c in enumerate(chars)) # numero - valore
print (char_indices)

{'\n': 0, ' ': 1, '!': 2, '"': 3, "'": 4, '(': 5, ')': 6, ',': 7, '-': 8, '.': 9, ':': 10, ';': 11, '<': 12, '>': 13, '?': 14, '`': 15, 'a': 16, 'b': 17, 'c': 18, 'd': 19, 'e': 20, 'f': 21, 'g': 22, 'h': 23, 'i': 24, 'j': 25, 'l': 26, 'm': 27, 'n': 28, 'o': 29, 'p': 30, 'q': 31, 'r': 32, 's': 33, 't': 34, 'u': 35, 'v': 36, 'x': 37, 'y': 38, 'z': 39}


In [5]:
maxlen = 30 # frazione di caratteri ridondante da me scelta
step = 3 # distanza tra i dati forniti alla rete
next_chars = []
sentences = [] #array dove metterò le frasi

In [6]:
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('numero frasi:', len(sentences))

numero frasi: 186070


In [7]:
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool) # matrice (amo le matrici) di input
y = np.zeros((len(sentences), len(chars)), dtype=np.bool) # output rappresentati ogni parola
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

In [8]:
model = Sequential() 
model.add(LSTM (128, input_shape=(maxlen, len(chars)))) #ottimizzata per gpu gentilemente offerrta da google perchè sono povero
model.add(Dense(len(chars), activation='softmax')) # normale layer di neuroni connessi completamente
model.compile(loss='categorical_crossentropy', optimizer=RMSprop(lr=0.01))

In [45]:
def testAfterEpoch (epoch,_):
    print()
    print()
    print('**************Epoch: %d**************' % epoch)

    start_index = random.randint(0, len(text) - maxlen - 1)
    
  

    generated = ''
    sentence = text[start_index: start_index + maxlen] #crea un chunk di 30 caratteri a partire da un indice a caso
    generated += sentence
    print('*******************Frase di partenza*****************')
    print(sentence)
    print('*****************************************************')
    sys.stdout.write(generated)
    frase = generated +'\n'
    for i in range(400):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = np.argmax(preds)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char
        frase = frase + next_char
        sys.stdout.write(next_char)
        sys.stdout.flush()
    model.save(f'{epoch}{time.time()}.h5')
    with open(str (epoch) + '.txt','w') as f:
        f.write (frase)
    print()

In [46]:
print_callback = LambdaCallback(on_epoch_end=testAfterEpoch)
#model.fit (x,y,batch_size=2048,epochs=65,callbacks=[print_callback])

In [39]:
def random_prediction (lunghezza,model):
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated = ''
    sentence = text[start_index: start_index + maxlen] #crea un chunk di 30 caratteri a partire da un indice a caso
    generated += sentence
    print('*******************Frase di partenza*****************')
    print(sentence)
    print('*****************************************************')
    sys.stdout.write(generated)
    frase = generated +'\n'
    for i in range(lunghezza):
        x_pred = np.zeros((1, maxlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)[0]
        next_index = np.argmax(preds)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char
        frase = frase + next_char
        sys.stdout.write(next_char)
        sys.stdout.flush()
    print ()

In [19]:
random_prediction (1000,model)

*******************Frase di partenza*****************
nver' l'arene,
  queste del ge
*****************************************************
nver' l'arene,
  queste del gener per l'altro riterso,
  tanto che per l'altro riso regno,
  che la via mai nel sentime artito,
  e disse 'l poco in questa vellea
  de l'altro rivol che piu` altra schiava,
  e di me non si mara in me che piu` va rimo.

e se non si mosse i disse: <<perche' non si chiama
  parlar che 'l mondo il calder d'amor mi parla.

pero` che l'altro riso di che si che si che fu mara.

la` ' visso a l'altro raggio nel corno
  che si che con la vista che son discenta,
  come tuo nel peccator che parlare imprende
  li occhi suoi che per l'altra volta internare e di colpi
  de la prese di cose un poco impieghe,
  e disse 'l maestro con la sua vece,
  che per la via mai non si che si ch'a la scrista in corse al val sol crosto,
  per che l'un vecchior del ciel si ch'a la pianendo,
  che si chiama il ciel di coloro ascolti.

quelli e` a

In [22]:
new_model = load_model ('1578730326.6612554.h5')

In [28]:
random_prediction (400,new_model)

*******************Frase di partenza*****************

<<uscicci mai alcuno, o per s
*****************************************************

<<uscicci mai alcuno, o per sue ante
  de la sua vista di colui che 'l grando,
  e come con li occhi di trappa,
  che si mastra di colui che 'l potero scuoi
  che tu trarro del tempo e l'ardenta.

lo mio volem sola parlar di contrari
  che si mastra chi se' tra la mia
  la mia discende in che si che si che convenne,
  con la sua santa ch'io dico e di corpo sua fiere.

non siete chiamo si rispuose,
  che per altra vostra colpa

In [35]:
def trymodel (name,lunghezza = 400,times = 1):
    new_model = load_model (name)
    for _ in range (times):
        random_prediction (lunghezza,new_model)

In [41]:
trymodel ('1578732182.0066555.h5',times = 5)
    

*******************Frase di partenza*****************
ori.

li occhi di beatrice, ch
*****************************************************
ori.

li occhi di beatrice, che per mezzo
  che l'altra in che si che si che si che si che con la sua vera` segue
  di mia vista la corpe chiescia,

che la via mai non si che si ch'a la scrista in corse al val sol crosto,
  per che l'un vecchior del ciel si ch'a la pianendo,
  che si chiama il ciel di coloro ascolti.

quelli e` amavazia di la`, si` com'io dissi
  che si mara in su la spera siacchia,

come sanza veramenta di la
*******************Frase di partenza*****************
ne la cui virtute
  l'esser di
*****************************************************
ne la cui virtute
  l'esser di color che per l'altro rivola.

se tu me; e pero` che l'altro ritenno
  che tutto si` che 'l ver che si chiama
  di la` da la caladesta di colui
  che si chiama il ciel di coloro ascolti.

quelli e` amavazia di la`, si` com'io dissi
  che si mara in su la spera

In [47]:
model.fit (x,y,batch_size=2048,epochs=3,callbacks=[print_callback])

Epoch 1/3
186070/186070 [==============================] - 300s 2ms/step - loss: 1.1061


**************Epoch: 0**************
*******************Frase di partenza*****************
pecorelle a pascer caccia.

co
*****************************************************
pecorelle a pascer caccia.

con tanta si rispuose il sanza suole;
  ma per che si passo` le sua mente,
  con questa di colui che si fa congiunto.

io mi scender la sua parlar discende
  a la sua vostra crista e l'altro stano,

cosi` questo con quella che si lascia.

e quel che son di color che si chiama,

fatto di quel che son di color ch'i' disse un segno degna,
  con si` che si` di colui che si rispuose
  quella che si` che 
Epoch 2/3
186070/186070 [==============================] - 299s 2ms/step - loss: 1.1051


**************Epoch: 1**************
*******************Frase di partenza*****************
ta>>;
  per ch'elli, accumulan
*****************************************************
ta>>;
  per ch'elli, accumulan di la`

In [48]:
model.save ('final.h5')